In [8]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [9]:
import datasets
from datasets import load_dataset

ds = load_dataset("Open-Orca/OpenOrca")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
ds

DatasetDict({
    train: Dataset({
        features: ['id', 'system_prompt', 'question', 'response'],
        num_rows: 4233923
    })
})

In [30]:
from transformers import  AutoModelForSeq2SeqLM , GenerationConfig , AutoTokenizer
from tokenizers import Tokenizer

In [31]:
models=AutoModelForSeq2SeqLM.from_pretrained('google/flan-t5-base')


In [32]:
tokenizer=AutoTokenizer.from_pretrained('google/flan-t5-base')

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [35]:
example=[40]
dash_line = '-'.join('' for x in range(100))
for i , index in enumerate(example):
  ques=ds['train'][index]['question']
  response=ds['train'][index]['response']
input=tokenizer(ques,return_tensors='pt')
output=tokenizer.decode(models.generate(input['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)


In [38]:
print(dash_line)
print('Example ', i + 1)
print(dash_line)
print(f'INPUT PROMPT:\n{ques}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{response}')
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
What is the best order to watch the Star Wars series?
What were orders 1 through 65 in Star Wars?
Do those questions have the same meaning?
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
The best order to watch the Star Wars series depends on the preferences of the viewer, but there are a few popular viewing orders that work well in different situations. I will list three of them below:

1. Release Order:
This is the order in which the films were released in theaters, starting with the Original Trilogy followed by the Prequel Trilogy and then the Sequel Trilogy. This order captures the experience of watching the films as they were introduced to the world.

- Episode IV: A New Hope (1977)
-

In [42]:
example=[40]
prompt = ''
dash_line = '-'.join('' for x in range(100))
for i , index in enumerate(example):
  ques=ds['train'][index]['question']
  response=ds['train'][index]['response']

prompt+=f"""
Question: {ques}

Response: {response}


"""
ques=ds['train'][50]['question']
response2=ds['train'][50]['response']
prompt+=f"""
Question: {ques}

"""
input=tokenizer(prompt,return_tensors='pt')
output=tokenizer.decode(models.generate(input['input_ids'],max_new_tokens=50)[0],skip_special_tokens=True)

In [44]:
print(dash_line)
print('Example ', i + 1)
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{response2}')
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Question: What is the best order to watch the Star Wars series?
What were orders 1 through 65 in Star Wars?
Do those questions have the same meaning?

Response: The best order to watch the Star Wars series depends on the preferences of the viewer, but there are a few popular viewing orders that work well in different situations. I will list three of them below:

1. Release Order:
This is the order in which the films were released in theaters, starting with the Original Trilogy followed by the Prequel Trilogy and then the Sequel Trilogy. This order captures the experience of watching the films as they were introduced to the world.

- Episode IV: A New Hope (1977)
- Episode V: The Empire Strikes Back (1980)
- Episode VI: Return of the Jedi (1983)
- Episode I: The Ph

In [45]:
for i , index in enumerate(example):
  ques=ds['train'][index]['question']
  response=ds['train'][index]['response']

prompt+=f"""
Question: {ques}

Response: {response}


"""
ques=ds['train'][50]['question']
response2=ds['train'][50]['response']
prompt+=f"""
Question: {ques}

"""
input=tokenizer(prompt,return_tensors='pt')
output=tokenizer.decode(models.generate(input['input_ids'],generation_config=GenerationConfig(max_new_token=100,temperature=0.5))[0],skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to contr

In [46]:
print(dash_line)
print('Example ', i + 1)
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{response2}')
print(dash_line)
print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Question: What is the best order to watch the Star Wars series?
What were orders 1 through 65 in Star Wars?
Do those questions have the same meaning?

Response: The best order to watch the Star Wars series depends on the preferences of the viewer, but there are a few popular viewing orders that work well in different situations. I will list three of them below:

1. Release Order:
This is the order in which the films were released in theaters, starting with the Original Trilogy followed by the Prequel Trilogy and then the Sequel Trilogy. This order captures the experience of watching the films as they were introduced to the world.

- Episode IV: A New Hope (1977)
- Episode V: The Empire Strikes Back (1980)
- Episode VI: Return of the Jedi (1983)
- Episode I: The Ph

In [47]:
def filter(text):
    if isinstance(text, str):
        tokens = tokenizer.tokenize(text)
        return len(tokens) >= 100
    else:
        return False

In [ ]:
for i in range(len(ds['train'])):
  if filter(ds['train'][i]['question']):
    continue
  else:
    fd=remove_row(ds['train'],i)

In [56]:
def remove_row(dataset, index):
    # Create a list of indices to keep
    indices_to_keep = [i for i in range(len(dataset)) if i != index]
    # Select the rows to keep
    filtered_dataset = dataset.select(indices_to_keep)
    return filtered_dataset


In [67]:
pro=''
my_ques='What is lily ? '
res='Flower'
pro+=f"""
Question: {my_ques}
{res}
"""
my_ques2='What is butterfly ? '
pro+=f"""
Question: {my_ques2}

"""
input=tokenizer(pro,return_tensors='pt')
output=tokenizer.decode(models.generate(input['input_ids'],generation_config=GenerationConfig(max_new_token=100,temperature=0.5))[0],skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:515: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to contr

In [68]:
print(dash_line)
print(pro)
print(dash_line)
print(output)

---------------------------------------------------------------------------------------------------

Question: What is lily ? 
Flower

Question: What is butterfly ? 


---------------------------------------------------------------------------------------------------
butterfly is a genus of flowering plants.


In [72]:
fd

Dataset({
    features: ['id', 'system_prompt', 'question', 'response'],
    num_rows: 4233922
})